In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from collections import defaultdict
import pickle
import torch
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import SVC
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
import torchvision

In [3]:
def load_data(path, label):
    with open(path, 'rb') as iffile:
        features = pickle.load(iffile)
    layer = 'flatten'
    features = features[layer]
    features_matrix = None

    for i in range(len(features)):
        if isinstance(features_matrix, torch.Tensor):
            features_matrix = torch.cat((features_matrix, features[i]))
        else:
            features_matrix = features[i]

    features = None    
    features_matrix = torch.flatten(features_matrix, start_dim=1, end_dim=-1)
    y = np.empty(features_matrix.shape[0])
    y.fill(label)
    return [features_matrix, y]


In [10]:
def unison_shuffled_copies(x_ben, y_ben, x_adv, y_adv):
    X = np.concatenate((x_ben, x_adv), axis=0)
    y = np.concatenate((y_ben, y_adv), axis=0)
    assert len(X) == len(y)
    p = np.random.permutation(len(X))
    return X[p], y[p]

In [11]:
base_path = "/bigstor/zsarwar/SparseDNNs/MT_CIFAR10_full_10_d5f3f545a0883adb9c8f98e2a6ba4ac7/MT_Baseline_d2a45a4dd02a5e037e5954b82387e666"
def get_paths(base_path, attack_type):
    relu_dir = "ReLUs"
    train_benign = f"ReLUs_{attack_type}_train_benign.pkl"
    train_adversarial = f"ReLUs_{attack_type}_train_adversarial.pkl"
    test_benign = f"ReLUs_{attack_type}_test_benign.pkl"
    test_adversarial = f"ReLUs_{attack_type}_test_adversarial.pkl"

    train_benign = os.path.join(base_path, relu_dir, train_benign)
    train_adversarial = os.path.join(base_path, relu_dir,  train_adversarial)
    test_benign = os.path.join(base_path,  relu_dir, test_benign)
    test_adversarial = os.path.join(base_path,  relu_dir, test_adversarial)

    all_paths = [train_benign,train_adversarial,test_benign,test_adversarial]

    return all_paths


In [12]:
attack_type="DeepFool"
train_benign,train_adversarial,test_benign,test_adversarial = get_paths(base_path, attack_type)

In [13]:
train_benign = load_data(train_benign, 1)
train_adversarial = load_data(train_adversarial, 0)
test_benign = load_data(test_benign, 1)
test_adversarial = load_data(test_adversarial, 0)

In [14]:
min_train = min(train_benign[0].shape[0], train_adversarial[0].shape[0])
min_test = min(test_benign[0].shape[0], test_adversarial[0].shape[0])

In [15]:
# Subsample the larger sets

rand_indices = np.random.choice(train_benign[0].shape[0], size=min_train, replace=False)
train_benign[0] = train_benign[0][rand_indices] 
train_benign[1] = train_benign[1][rand_indices]

rand_indices = np.random.choice(test_benign[0].shape[0], size=min_test, replace=False)
test_benign[0] = test_benign[0][rand_indices] 
test_benign[1] = test_benign[1][rand_indices]


In [20]:
X_train, y_train = unison_shuffled_copies(train_benign[0], train_benign[1], train_adversarial[0], train_adversarial[1])

X_test, y_test = unison_shuffled_copies(test_benign[0], test_benign[1], test_adversarial[0], test_adversarial[1])


In [23]:
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.svm import SVC

clf = SVC(C=0.7, gamma=0.075)
clf.fit(X_train, y_train)

SVC(C=0.7, gamma=0.075)

In [31]:
# Testing
x_train_pred = clf.predict(X_train)
train_acc = len(np.where(x_train_pred == y_train)[0]) / len(X_train)

x_test_pred = clf.predict(X_test)
test_acc = len(np.where(x_test_pred == y_test)[0]) / len(X_test)

print("Training acc : ", train_acc)
print("Test acc : ", test_acc)

In [36]:
# Test classwise
x_train_ben_pred = clf.predict(train_benign[0])
train_ben_acc = len(np.where(x_train_ben_pred == train_benign[1])[0]) / len(x_train_ben_pred)

x_train_adv_pred = clf.predict(train_adversarial[0])
train_adv_acc = len(np.where(x_train_adv_pred == train_adversarial[1])[0]) / len(x_train_adv_pred)




x_test_ben_pred = clf.predict(test_benign[0])
test_ben_acc = len(np.where(x_test_ben_pred == test_benign[1])[0]) / len(x_test_ben_pred)


x_test_adv_pred = clf.predict(test_adversarial[0])
test_adv_acc = len(np.where(x_test_adv_pred == test_adversarial[1])[0]) / len(x_test_adv_pred)


In [37]:
print("Training benign acc : ", train_ben_acc)
print("Training adversarial acc : ", train_adv_acc)
print("-------------------------------------")
print("Testing benign acc : ", test_ben_acc)
print("Testing adversarial acc : ", test_adv_acc)

Training benign acc :  0.958984375
Training adversarial acc :  0.9421875
-------------------------------------
Testing benign acc :  0.8876953125
Testing adversarial acc :  0.9466796875
